# PyTorch Dataset

In [64]:
import os
import sys
from importlib import reload
sys.path.append('../src')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

In [65]:
import transformers

In [2]:
import torch.utils.data as D

In [63]:
import dataset

In [61]:
class Dataset(D.Dataset):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn
        self.dataset = np.load(fn)
        self.x = self.dataset['x']
        self.y = self.dataset['y']
        
    def process(self, x, y):
        return x, y
        
    def __getitem__(self, i):
        x, y = self.x[i], self.y[i]
        return self.process(x, y)
    
    def __len__(self):
        return len(self.y)

In [62]:
valid = Dataset('../input/validation.npz')

In [10]:
tokenizer = transformers.XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')

## Augmentations

Shuffle sentences

In [59]:
class BaseAugmentation:
    def __init__(self, p=0.5, bos_token=0, pad_token=1, eos_token=2, dot_token=5):
        self.bos_token = bos_token
        self.pad_token = pad_token
        self.eos_token = eos_token
        self.dot_token = dot_token
        self.p = p
    
    def __call__(self, *args, **kwargs):
        raise NotImplementedError

In [60]:
class ShuffleSentences(BaseAugmentation):
    """
    Shuffles sentences after splitting by dot token
    """
        
    def __call__(self, sent):
        new_sent = sent.copy()
        
        if np.random.uniform() < self.p:
            # Mask meaningful part of a sentence
            mask = (sent != self.bos_token) & (sent != self.pad_token) & (sent != self.eos_token)
            # Localize dots
            # TODO: a smatrer way to split?
            dots = np.where(sent == self.dot_token)[0]
            # Split by dot localization, shuffle
            split = np.split(sent[mask], dots)
            np.random.shuffle(split)
            # Concatenate back and paste
            new_sent[mask] = np.concatenate(split)
        
        return new_sent

In [57]:
shuffle = ShuffleSentences(1)

In [78]:
%timeit _ = shuffle(valid[12][0])

24.8 µs ± 5.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [77]:
tokenizer.decode(shuffle(valid[12][0]))

'<s> No sabemos quien es el autor del pie de imagen por ende no podemos acusarlo de haber inventado o no inventado la afirmacion, ergo no podemos decir que es investigacion original ya que no sabemos de donde obtuvo las lineas que escribió, lo que si sabemos es que la carga de la prueba recae sobre quien hace la afirmación, y en este caso es Xana la que afirma que el pie de imagen es subjetivo, pero, ¿segun quien?¿segun ella misma?, entonces en ese caso decir que el pie de iamgen es subjetivo es una afirmacion subjetiva de la propia subjetividad de Xana ya que no ha demostrado pruebas objetivas de que la susodicha sea realmente subjetiva, ergo eliminar tanto el pie de foto como la imagen misma nomás porque así lo dice la subjetividad de Xana, eso sería precisamente una edición subjetiva carente de argumentacion enciclopédica y por eso procedí a revertirla.Mr.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [76]:
tokenizer.decode(valid[12][0])

'<s> No sabemos quien es el autor del pie de imagen por ende no podemos acusarlo de haber inventado o no inventado la afirmacion, ergo no podemos decir que es investigacion original ya que no sabemos de donde obtuvo las lineas que escribió, lo que si sabemos es que la carga de la prueba recae sobre quien hace la afirmación, y en este caso es Xana la que afirma que el pie de imagen es subjetivo, pero, ¿segun quien?¿segun ella misma?, entonces en ese caso decir que el pie de iamgen es subjetivo es una afirmacion subjetiva de la propia subjetividad de Xana ya que no ha demostrado pruebas objetivas de que la susodicha sea realmente subjetiva, ergo eliminar tanto el pie de foto como la imagen misma nomás porque así lo dice la subjetividad de Xana, eso sería precisamente una edición subjetiva carente de argumentacion enciclopédica y por eso procedí a revertirla.Mr.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>